In [1]:
import pandas as pd
import os
#cerrega todo os arquivos
dfsNew = []
dfsOld = []

for arq in os.listdir("./data/ES/"):
    if arq.find("New") > 0:
        dfsNew.append(pd.read_csv("./data/ES/"+arq, sep="\t"))
    elif arq.find("Old") > 0:
        dfsOld.append(pd.read_csv("./data/ES/"+arq, sep="\t"))


In [2]:
from nltk import ngrams
from sacremoses import MosesTokenizer

def tokenize(text, lang, nGram=3):
    entok = MosesTokenizer(lang=lang)
    text = entok.tokenize(text, escape=False)
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram, lang):

  overlaps = []
  for h, r in zip([hypothesys], [references]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, lang, nGram)
    b = tokenize(r, lang, nGram)

    if len(a) >= len(b):
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a):
      overlaps.append(len(set(a) & set(b))/len(b))

  return overlaps[0]

import numpy as np

def getStats(dfA, dfB, lang):
    df = dfA.merge(dfB, on=["livro", "capitulo", "versiculo"])

    df["sourceLen"] = df["texto_x"].apply(lambda x: getSizeSentece(x))
    df["targetLen"] = df["texto_y"].apply(lambda x: getSizeSentece(x))
    
    df["overlap"] = df.apply(lambda x: getNgramOverlap(x["texto_x"], x["texto_y"], 3, lang), axis=1)

    return df
#Matriz do novo testamento

def getSizeSentece(text):
    try:
        return len(text.split(" "))
    except:
        return 0
    


In [3]:
print("Novo testamento: ")
newConcat = []
for dfA in dfsNew:
    dfsNew = dfsNew[1:]
    for dfB in dfsNew:
        newConcat.append(getStats(dfA, dfB, "es"))

Novo testamento: 


In [7]:
dfConcat = pd.concat(newConcat, ignore_index=False)
print("tamanho sem cortes: ", dfConcat.shape)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]
#filtra sentencas curtas
dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho com cortes: ", dfConcat.shape)

tamanho sem cortes:  (627354, 10)
tamanho com cortes:  (602095, 10)


In [8]:
dfConcat.to_csv("./data/ES/NovoTestamentoCompleto-ES.tsv", sep="\t", index=False)

In [3]:
print("Velho testamento: ")
oldConcat = []
for dfA in dfsOld:
    dfsOld = dfsOld[1:]
    for dfB in dfsOld:
        oldConcat.append(getStats(dfA, dfB, "es"))

Velho testamento: 


In [4]:
dfConcat = pd.concat(oldConcat, ignore_index=False)
print("tamanho sem cortes: ", dfConcat.shape)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]
#filtra sentencas curtas
dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho com cortes: ", dfConcat.shape)

tamanho sem cortes:  (1993269, 10)
tamanho com cortes:  (1888623, 10)


In [5]:
dfConcat.to_csv("./data/ES/VelhoTestamentoCompleto-ES.tsv", sep="\t", index=False)

In [13]:
dfConcat

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap
0,BLPH,Génesis,1,1,"Cuando Dios , en el principio , creó los cielo...",BLP,"Cuando Dios , en el principio , creó los cielo...",14,14,0.925926
1,BLPH,Génesis,1,2,la tierra era una masa caótica y las tinieblas...,BLP,la tierra era una masa caótica y las tinieblas...,24,24,0.957447
2,BLPH,Génesis,1,3,Entonces dijo Dios : — ¡Que exista la luz ! Y ...,BLP,Entonces dijo Dios : — ¡Que exista la luz ! Y ...,15,15,0.903226
3,BLPH,Génesis,1,4,"Al ver Dios que la luz era buena , la separó d...",BLP,"Al ver Dios que la luz era buena , la separó d...",15,15,0.931034
4,BLPH,Génesis,1,5,"llamando a la luz “ día ” y a las tinieblas , ...",BLP,"llamando a la luz “ día ” y a las tinieblas , ...",30,30,0.847458
...,...,...,...,...,...,...,...,...,...,...
16681,SRV-BRG,Malaquías,4,4,"Acordaos de la ley de Moisés mi siervo , al cu...",TLA,"» Israelitas , recuerden que en el monte Horeb...",21,27,0.145455
16682,SRV-BRG,Malaquías,4,2,"Mas á vosotros los que teméis mi nombre , nace...",TLA,"» Para ustedes , que me respetan , la justicia...",33,35,0.173913
16683,SRV-BRG,Malaquías,4,3,"Y hollaréis á los malos , los cuales serán cen...",TLA,"Ya tengo preparado ese día , y ustedes aplasta...",31,35,0.086957
16684,SRV-BRG,Malaquías,4,5,"He aquí , yo os envío á Elías el profeta , ant...",TLA,» Antes de que llegue el día en que yo los cas...,22,19,0.232558
